In [1]:
def preludeF(vector):
    num = len(vector)
    trans_vector = []
    for i in range(num):
        target = vector[i]
        if target == 1:
            trans_vector.append(4)
        elif target == 3:
            trans_vector.append(6)
        elif target == 5:
            trans_vector.append(8)
        elif target == 7:
            trans_vector.append(12)
    return trans_vector

def preludeG(vector):
    num = len(vector)
    trans_vector = []
    for i in range(num):
        target = vector[i]
        if target == 1:
            trans_vector.append(5)
        elif target == 3:
            trans_vector.append(7)
        elif target == 5:
            trans_vector.append(9)
        elif target == 7:
            trans_vector.append(13)
    return trans_vector

def preludeAm(vector):
    num = len(vector)
    trans_vector = []
    for i in range(num):
        target = vector[i]
        if target == 1:
            trans_vector.append(6)
        elif target == 3:
            trans_vector.append(8)
        elif target == 5:
            trans_vector.append(10)
        elif target == 7:
            trans_vector.append(12)
    return trans_vector

def preludeC(vector):
    num = len(vector)
    trans_vector = []
    for i in range(num):
        target = vector[i]
        if target == 1:
            trans_vector.append(1)
        elif target == 3:
            trans_vector.append(3)
        elif target == 5:
            trans_vector.append(5)
        elif target == 7:
            trans_vector.append(2)
    return trans_vector

In [2]:
# 1 3 5 分别代表根音、三音、五音。7分别用在大调和小调中对应9和7
# 和弦走向位F、G、Am、Am，其中前三项保持一致，最后一个Am独立生成

In [3]:
import numpy as np
import random

In [4]:
Clist = [1,3,5,7]
repeat_sequence = np.random.choice(Clist,7,p=[1/3,1/4,1/4,1/6])
repeat_sequence = np.insert(repeat_sequence,0,1)
Am_sequence = np.random.choice(Clist,7,p=[1/3,1/4,1/4,1/6])
Am_sequence = np.insert(Am_sequence,0,1)
C_sequence = np.random.choice(Clist,7,p=[1/3,1/4,1/4,1/6])
C_sequence = np.insert(C_sequence,0,1)

In [5]:
#和旋计划走向是F->G->Am->Am->F->G->AM->C（一共八组）

In [6]:
prelude_seq = []
i = 0
for i in range(8):
    if i%8 == 0:    #F
        prelude_seq.extend(preludeF(repeat_sequence))
    elif i%8 == 1:    #G
        prelude_seq.extend(preludeG(repeat_sequence))
    elif i%8 == 2:    #Am
        prelude_seq.extend(preludeAm(repeat_sequence))
    elif i%8 == 3:    #Am
        prelude_seq.extend(preludeAm(Am_sequence))
    elif i%8 == 4:    #F
        prelude_seq.extend(preludeF(repeat_sequence))
    elif i%8 == 5:    #G
        prelude_seq.extend(preludeG(repeat_sequence))
    elif i%8 == 6:    #Am
        prelude_seq.extend(preludeAm(repeat_sequence))
    elif i%8 == 7:    #C
        prelude_seq.extend(preludeC(C_sequence))

print(prelude_seq)

[4, 6, 6, 12, 4, 6, 6, 6, 5, 7, 7, 13, 5, 7, 7, 7, 6, 8, 8, 12, 6, 8, 8, 8, 6, 8, 8, 6, 6, 8, 8, 10, 4, 6, 6, 12, 4, 6, 6, 6, 5, 7, 7, 13, 5, 7, 7, 7, 6, 8, 8, 12, 6, 8, 8, 8, 1, 1, 2, 3, 5, 2, 5, 5]


In [7]:
#当在60以后的时候，123456对应的mido表示
import math

def map_remain(num):
    if num == 1:
        result = 1
    elif num == 1.5:
        result = 2
    elif num == 2:
        result = 3
    elif num == 2.5:
        result = 4
    elif num == 3:
        result = 5
    elif num == 4:
        result = 6
    elif num == 4.5:
        result = 7
    elif num == 5:
        result = 8
    elif num == 5.5:
        result = 9
    elif num == 6:
        result = 10
    elif num == 6.5:
        result = 11
    elif num == 0:
        result = 0
    return result        
        
        
def toMidi(vector):
    
    num = len(vector)
    trans_vector = []
    for i in range(num):
        base = 59   #基数是59
        target = vector[i]
        int_part = math.floor(target)     #得到target的整数部分
        times = math.floor(int_part/7)    #整数部分是由多少个7组成的
        base += times*12
        
        remain = target - times*7        #得到剩余部分
        base += map_remain(remain)
        trans_vector.append(base)
    return trans_vector

midi_prelude_seq = toMidi(prelude_seq)
print(midi_prelude_seq)
len(midi_prelude_seq)

[65, 69, 69, 79, 65, 69, 69, 69, 67, 71, 71, 81, 67, 71, 71, 71, 69, 72, 72, 79, 69, 72, 72, 72, 69, 72, 72, 69, 69, 72, 72, 76, 65, 69, 69, 79, 65, 69, 69, 69, 67, 71, 71, 81, 67, 71, 71, 71, 69, 72, 72, 79, 69, 72, 72, 72, 60, 60, 62, 64, 67, 62, 67, 67]


64

In [8]:
import mido
from mido import Message, MidiFile, MidiTrack

In [9]:
mid = MidiFile()       #给自己的文件定的.mid后缀
track = MidiTrack()    #定义声部，一个MidoTrack()就是一个声部

In [10]:
def beat(time_beat):            #与mido的拍子互换
    miao = 60/time_beat
    time = miao*1000
    return time

def yin(yin,pai,qian=0,unit = track, liang=64,qi=2): #yin是音，pai是指时间（节拍），
    if type(yin)== str:               #qian是是音前多少拍
        yin = num(yin)                #liang是音量

    unit.append(Message('program_change',channel=0,program=qi,time=0))
    unit.append(Message('note_on',note=yin,velocity=liang,time=qian,channel=0))
    unit.append(Message('note_off',note=yin,velocity=liang,time=int(beat(pai)),channel=0))

In [11]:
j = 0
for j in range(64):    #16个小节，每小节8拍，一共128
    yin(midi_prelude_seq[j],160, 0,track,80,2)

In [12]:
mid.tracks.append(track)
mid.save('C:\\Users\\Administrator\\Desktop\\2020Summer\\肠粉4.mid')